In [1]:
import itertools
from functools import partial

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from cycler import cycler

In [2]:
def filled_histogram(ax, edges, values, bottoms=None, orientation="v", **kwargs):
    """Draw a histogram as a stepped patch.
    
    Parameters
    ----------
    ax : Axes
        The axes to plot to
    
    edges : array
        A length n+1 array giving the left edges of each bin and the
        right edge of the last bin.
    
    values : array
        A length n array of bin counts or values
    
    bottoms : float or array, optional
        A length n array of the bottom of the bars.  If None, zero is used.
    
    orientation : {'v', 'h'}
        Orientation of the histogram.  'v' (default) has
        the bars increasing in the positive y-direction.
    
    **kwargs
        Extra keyword arguments are passed through to `.fill_between`.
    
    Returns
    -------
    ret : PolyCollection
        Artist added to the Axes
    """
    print(orientation)
    if orientation not in "hv":
        raise ValueError(f"orientation must be in {{'h', 'v'}}, not {orientation}")
    
    kwargs.setdefault("step", "post")
    kwargs.setdefault("alpha", 0.7)
    edges = np.asarray(edges)
    values = np.asarray(values)
    
    if len(edges) - 1 != len(values):
        raise ValueError(f"Must provide one more bin edge than value not: {len(edges)=} {len(values)=}")
    
    if bottoms is None:
        bottoms = 0
    bottoms = np.broadcast_to(bottoms, values.shape)
    
    values = np.append(values, values[-1])
    bottoms = np.append(bottoms, bottoms[-1])
    
    if orientation == "h":
        return ax.fill_betweenx(edges, values, bottoms, **kwargs)
    elif orientation == "v":
        return ax.fill_between(edges, values, bottoms, **kwargs)
    else:
        raise AssertionError("you should never be here")

In [3]:
def stack_histogram(
        ax, stacked_data, sty_cycle, bottoms=None, hist_function=None,
        labels=None, plot_function=None, plot_kwargs=None):
    """
    Parameters
    ----------
    ax : axes.Axes
        The axes to add artists too
    
    stacked_data : array or Mapping
        A (M, N) shaped array.  The first dimension will be iterated over to
        compute histograms row-wise
    
    sty_cycle : Cycler or operable of dict
        Style to apply to each set
    
    bottoms : array, default: 0
        The initial positions of the bottoms.
    
    hist_func : callable, optional
        Must have signature `bin_vals, bin_edges = f(data)`.
        `bin_edges` expected to be one longer than `bin_vals`
    
    labels : list of str, optional
        The label for each set.
        
        If not given and stacked data is an array defaults to 'default set {n}'
        
        If *stacked_data* is a mapping, and *labels* is None, default to the
        keys.
        
        If *stacked_data* is a mapping and *labels* is given then only the
        columns listed will be plotted.
    
    plot_func : callable, optional
        Function to call to draw the histogram must have signature:
        
        ret = plot_func(ax, edges, top, bottoms=bottoms,
                        label=label, **kwargs)
    
    plot_kwargs : dict, optional
        Any extra keyword arguments to pass through to the plotting function.
        This will be the same for all calls to the plotting function and will
        override the values in *sty_cycle*.
    
    Returns
    -------
    arts : dict
        Dictionary of artists keyed on their labels
    """
    # deal with default binning function
    if hist_function is None:
        hist_function = np.histogram
    
    # deal with default plotting function
    if plot_function is None:
        plot_function = filled_histogram
    
    # deal with default
    if plot_kwargs is None:
        plot_kwargs = {}
    print(plot_kwargs)
    
    try:
        l_keys = stacked_data.keys()
        label_data = True
        if labels is None:
            labels = l_keys
    except AttributeError:
        label_data = False
        if labels is None:
            labels = itertools.repeat(None)
    
    if label_data:
        loop_iter = enumerate(
            (stacked_data[label], label, s) for label, s in zip(labels, sty_cycle)
        )
    else:
        loop_iter = enumerate(zip(stacked_data, labels, sty_cycle))
    
    arts = {}
    for j, (data, label, sty) in loop_iter:
        if label is None:
            label = f"default set {j}"
        
        label = sty.pop("label", label)
        values, edges = hist_function(data)
        
        if bottoms is None:
            bottoms = np.zeros_like(values)
        
        top = bottoms + values
        print(f"Before: {sty=}")
        sty.update(plot_kwargs)
        print(f"After: {sty=}")
        
        ret = plot_function(ax, edges, top, bottoms=bottoms, label=label, **sty)
        bottoms = top
        arts[label] = ret
    
    ax.legend(fontsize=10)
    return arts